<a href="https://colab.research.google.com/github/ola-sumbo/Hamoye-Internship/blob/master/Linear_classification_and_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import numpy as np
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')
# check the distribution of target variable by doing a count
df['QScore'].value_counts()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [135]:
# check for missing values and if any drop rows that are not necessary
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [136]:
df= df.dropna() # will drop missing values row-wise

An obvious change in our target variable after removing the missing values is that there are only three classes left #and from the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. #There are methods that can be applied to handle this imbalance such as oversampling and **undersampling**.
#Oversampling involves increasing the number of instances in the class with fewer instances while undersampling involves reducing the data points in the class with more instances.
For now, we will convert this to a binary classification problem by combining class '2A' and '1A'.

In [137]:
df['QScore'].value_counts() # check to verify

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [138]:
df['QScore'] = df['QScore'].replace(['1A'],'2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [139]:
df_2A = df[df.QScore == '2A']
df_3A = df[df.QScore == '3A'].sample(350)
data_df = df_2A.append(df_3A) # adds df_3a df to df_2A

In [140]:
import sklearn.utils 
data_df = sklearn.utils.shuffle(data_df) # data shuffle from algeria to ethiopia, notice the country column
data_df = data_df.reset_index(drop=True) # resets the index to default and if it is multilevel it removes one or more levels
data_df.shape
data_df['QScore'].value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [141]:
# more preprocessing 
data_df = data_df.drop(columns=['country_code','country','year'])
X = data_df.drop(columns='QScore')
y = data_df['QScore']

In [142]:
# split data into training and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =  train_test_split(X, y, test_size = 0.3, random_state =0)
y_train.value_counts() # there is still an imbalance in the dataset, hence we will use SMOTE to handle this on training dataset only

3A    245
2A    168
Name: QScore, dtype: int64

In [143]:
# encode the categorical variable which only exists in this dataset as records
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record) # from data x_train select column record
x_test.record = encoder.transform(x_test.record)

import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [144]:
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
scaler = MinMaxScaler()
normalised_train_df = pd.DataFrame(x_train_balanced, columns=x_train.columns)
normalised_train_df = scaler.fit_transform(normalised_train_df.drop(columns=['record']))

x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [146]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

#returns


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)